<p style='color:red ; font-size:32px; text-align:center;' dir=rtl>
سوال دوم 
<br>
قسمت سوم : تحلیل احساس نظرات
<br>
با استفاده از مدل RoBERTa

<p style='color:yellow ; font-size:28px; text-align:center;' dir=rtl>
وارد کردن توابع مورد نیاز

In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from transformers import RobertaTokenizer, RobertaForSequenceClassification 
from tqdm.auto import tqdm
from dataclasses import dataclass

c:\Users\M.H.A\anaconda3\envs\sentiment_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<p style='color:yellow ; font-size:28px; text-align:center;' dir=rtl>
1. تنظیمات عمومی

In [ ]:
# --- 1. تنظیمات عمومی ---
MODEL_NAME = 'roberta-base' # 🌟 تغییر اصلی: نام مدل به roberta-base
MAX_LEN = 128                 
BATCH_SIZE = 16               
NUM_EPOCHS = 3                
LEARNING_RATE = 2e-5          

# --- تشخیص دستگاه (CPU یا GPU) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"استفاده از دستگاه: {device}")
if device.type == 'cuda':
    print(f"نام GPU: {torch.cuda.get_device_name(0)}")
    print(f"حافظه GPU (کلی): {torch.cuda.get_device_properties(0).total_memory / (1024**3):.2f} GB")

tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME) 
model = RobertaForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=5) 
model.to(device) # انتقال مدل به GPU (اگر موجود باشد)

# --- کلاس برای نگهداری ورودی‌های توکن‌سازی شده ---
@dataclass
class EncodedData:
    input_ids: torch.Tensor
    attention_mask: torch.Tensor
    labels: torch.Tensor = None 

استفاده از دستگاه: cuda
نام GPU: NVIDIA GeForce RTX 2060
حافظه GPU (کلی): 6.00 GB


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<p style='color:yellow ; font-size:28px; text-align:center;' dir=rtl>
2. کلاس Dataset برای داده‌های آموزش (با لیبل) 

In [3]:
# --- 2. کلاس Dataset برای داده‌های آموزش (با لیبل) ---
class TrainReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings 
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        return {
            'input_ids': self.encodings['input_ids'][item].flatten(),
            'attention_mask': self.encodings['attention_mask'][item].flatten(),
            'labels': torch.tensor(self.labels[item] - 1, dtype=torch.long)
        }

<p style='color:yellow ; font-size:28px; text-align:center;' dir=rtl>
3. کلاس Dataset برای داده‌های پیش بینی (بدون لیبل) 

In [4]:
# --- 3. کلاس Dataset برای داده‌های پیش‌بینی (بدون لیبل) ---
class PredictReviewDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, item):
        return {
            'input_ids': self.encodings['input_ids'][item].flatten(),
            'attention_mask': self.encodings['attention_mask'][item].flatten(),
        }


<p style='color:yellow ; font-size:28px; text-align:center;' dir=rtl>
4. تابع بارگذاری و پیش‌پردازش داده از فایل‌ها 

In [ ]:
# --- 4. تابع بارگذاری و پیش‌پردازش داده از فایل‌ها ---
def load_and_preprocess_data_for_training_and_prediction(train_file_path, predict_file_path, text_col='reviewText', target_col='overall'):
    try:
        temp_train_df = pd.read_csv(train_file_path)
        
        #  30000 سطر تصادفی از آن را انتخاب می‌کنیم
        # اگر تعداد کل سطرها کمتر از 30000 باشد، همه سطرها انتخاب می‌شوند.
        train_df = temp_train_df.sample(n=min(30000, len(temp_train_df)), random_state=42) 
        predict_df = pd.read_csv(predict_file_path) 
        
        print(f"✅ داده‌های آموزش از '{train_file_path}' با موفقیت بارگذاری شد. تعداد ردیف‌ها: {len(train_df)}")
        print(f"✅ داده‌های پیش‌بینی از '{predict_file_path}' با موفقیت بارگذاری شد. تعداد ردیف‌ها: {len(predict_df)}")
        
        train_df.dropna(subset=[text_col, target_col], inplace=True)
        train_df[text_col] = train_df[text_col].astype(str)
        train_df[target_col] = train_df[target_col].astype(int)
        
        predict_df.dropna(subset=[text_col], inplace=True) 
        predict_df[text_col] = predict_df[text_col].astype(str)
        
        return train_df, predict_df
    except FileNotFoundError:
        print(f"❌ خطا: یکی از فایل‌ها یافت نشد. لطفاً مسیر فایل‌ها را بررسی کنید.")
        return None, None
    except Exception as e:
        print(f"❌ خطا در بارگذاری یا پیش‌پردازش فایل: {e}")
        return None, None

 <p style='color:yellow ; font-size:28px; text-align:center;' dir=rtl>
تابع جدید: توکن‌سازی دسته‌ای در ابتدای کار


In [6]:
# --- تابع جدید: توکن‌سازی دسته‌ای در ابتدای کار ---
def tokenize_data_batch(texts, tokenizer, max_len):
    print("\n⏳ در حال توکن‌سازی داده‌ها (این ممکن است کمی طول بکشد)...")
    encodings = tokenizer.batch_encode_plus(
        texts.tolist(), 
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
    )
    print("✅ توکن‌سازی داده‌ها با موفقیت انجام شد.")
    return encodings

 <p style='color:yellow ; font-size:28px; text-align:center;' dir=rtl>
 5. تابع ساخت DataLoaderها (با داده‌های از پیش توکن‌سازی شده) 

In [ ]:
# --- 5. تابع ساخت DataLoaderها (با داده‌های از پیش توکن‌سازی شده) ---
def create_data_loaders(train_df, predict_df, tokenizer, max_len, batch_size):
    if train_df is None or predict_df is None:
        return None, None

    train_encodings = tokenize_data_batch(train_df.reviewText, tokenizer, max_len)
    predict_encodings = tokenize_data_batch(predict_df.reviewText, tokenizer, max_len)

    train_dataset = TrainReviewDataset(
        encodings=train_encodings,
        labels=train_df.overall.to_numpy()
    )

    predict_dataset = PredictReviewDataset(
        encodings=predict_encodings
    )

    train_data_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0 
    )

    predict_data_loader = DataLoader(
        predict_dataset,
        batch_size=batch_size,
        num_workers=0 
    )
    
    print(f"\nتعداد دسته‌های آموزش: {len(train_data_loader)}")
    print(f"تعداد دسته‌های پیش‌بینی: {len(predict_data_loader)}")
    return train_data_loader, predict_data_loader


 <p style='color:yellow ; font-size:28px; text-align:center;' dir=rtl>
 6. تابع آموزش یک Epoch 


In [8]:
# --- 6. تابع آموزش یک Epoch ---
def train_epoch(model, data_loader, optimizer, device):
    model.train() 
    losses = []
    correct_predictions = 0
    total_samples = 0

    for d in tqdm(data_loader, desc="Training"):
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels = d["labels"].to(device)

        optimizer.zero_grad() 

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels 
        )

        loss = outputs.loss
        logits = outputs.logits
        
        _, preds = torch.max(logits, dim=1)
        
        correct_predictions += torch.sum(preds == labels)
        total_samples += labels.size(0)

        loss.backward() 
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) 
        optimizer.step() 
        losses.append(loss.item())

    return correct_predictions.double() / total_samples, sum(losses) / len(losses)


 <p style='color:yellow ; font-size:28px; text-align:center;' dir=rtl>
7. تابع پیش‌بینی روی داده‌های جدید (بدون لیبل) 

In [9]:
# --- 7. تابع پیش‌بینی روی داده‌های جدید (بدون لیبل) ---
def predict_on_new_data(model, data_loader, device):
    model.eval() 
    all_predictions = []
    
    with torch.no_grad(): 
        for d in tqdm(data_loader, desc="Predicting"):
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            
            logits = outputs.logits
            _, preds = torch.max(logits, dim=1)
            
            all_predictions.extend(preds.cpu().numpy())

    return [p + 1 for p in all_predictions] 



 <p style='color:yellow ; font-size:28px; text-align:center;' dir=rtl>
 8. تابع اصلی برای اجرای همه مراحل 

In [10]:
# --- 8. تابع اصلی برای اجرای همه مراحل ---
def run_deep_learning_sentiment_prediction(train_file_path, predict_file_path, text_col='reviewText', target_col='overall'):
    train_df, predict_df = load_and_preprocess_data_for_training_and_prediction(train_file_path, predict_file_path, text_col=text_col, target_col=target_col)

    if train_df is None or predict_df is None:
        print("🚨 بارگذاری داده‌ها با خطا مواجه شد. برنامه متوقف می‌شود.")
        return

    train_data_loader, predict_data_loader = create_data_loaders(
        train_df, predict_df, tokenizer, MAX_LEN, BATCH_SIZE
    )

    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)

    print("\n--- شروع آموزش مدل ---")
    for epoch in range(NUM_EPOCHS):
        print(f"\n--- Epoch {epoch + 1}/{NUM_EPOCHS} ---")
        train_acc, train_loss = train_epoch(
            model,
            train_data_loader,
            optimizer,
            device
        )
        print(f"✨ آموزش - دقت: {train_acc:.4f}, خطا: {train_loss:.4f}")

    print("\n--- آموزش مدل به پایان رسید. شروع پیش‌بینی روی داده‌های جدید ---")
    
    predicted_ratings = predict_on_new_data(model, predict_data_loader, device)

    submission_df = pd.DataFrame({'predicted': predicted_ratings})
    
    output_file_path = 'q2_submission.csv'
    submission_df.to_csv(output_file_path, index=False) 

    print(f"\n🎉 پیش‌بینی‌ها با موفقیت در فایل '{output_file_path}' ذخیره شدند.")
    print("📋 ۱۰ ردیف اول فایل ارسالی:")
    print(submission_df.head(10))


 <p style='color:yellow ; font-size:28px; text-align:center;' dir=rtl>
 اجرای اسکریپت نهایی

In [11]:
# --- اجرای اسکریپت ---
if __name__ == "__main__":
    train_csv_file_path = 'train_data.csv' 
    predict_csv_file_path = 'test_data.csv' 
    
    run_deep_learning_sentiment_prediction(
        train_csv_file_path, 
        predict_csv_file_path, 
        text_col='reviewText', 
        target_col='overall'
    )

D:\AppData\ipykernel_7420\1440439958.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_train_df = pd.read_csv(train_file_path)


✅ داده‌های آموزش از 'train_data.csv' با موفقیت بارگذاری شد. تعداد ردیف‌ها: 30000
✅ داده‌های پیش‌بینی از 'test_data.csv' با موفقیت بارگذاری شد. تعداد ردیف‌ها: 20000

⏳ در حال توکن‌سازی داده‌ها (این ممکن است کمی طول بکشد)...
✅ توکن‌سازی داده‌ها با موفقیت انجام شد.

⏳ در حال توکن‌سازی داده‌ها (این ممکن است کمی طول بکشد)...
✅ توکن‌سازی داده‌ها با موفقیت انجام شد.

تعداد دسته‌های آموزش: 1875
تعداد دسته‌های پیش‌بینی: 1250

--- شروع آموزش مدل ---

--- Epoch 1/3 ---


Training: 100%|██████████| 1875/1875 [18:04<00:00,  1.73it/s]


✨ آموزش - دقت: 0.6781, خطا: 0.8164

--- Epoch 2/3 ---


Training: 100%|██████████| 1875/1875 [18:17<00:00,  1.71it/s]


✨ آموزش - دقت: 0.7353, خطا: 0.6618

--- Epoch 3/3 ---


Training: 100%|██████████| 1875/1875 [18:02<00:00,  1.73it/s]


✨ آموزش - دقت: 0.7785, خطا: 0.5641

--- آموزش مدل به پایان رسید. شروع پیش‌بینی روی داده‌های جدید ---


Predicting: 100%|██████████| 1250/1250 [02:13<00:00,  9.36it/s]



🎉 پیش‌بینی‌ها با موفقیت در فایل 'q2_submission.csv' ذخیره شدند.
📋 ۱۰ ردیف اول فایل ارسالی:
   predicted
0          1
1          1
2          1
3          1
4          1
5          1
6          1
7          1
8          1
9          2


In [ ]:
# --- کد ذخیره مدل در اینجا اضافه می‌شود ---
# نام پوشه‌ای که مدل در آن ذخیره می‌شود
output_dir = "roberta_sentiment_model_trained" 

# مطمئن می‌شویم که پوشه خروجی وجود دارد
import os
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
# ذخیره مدل (وزن‌ها و پیکربندی)
model_to_save = model.module if hasattr(model, 'module') else model 
model_to_save.save_pretrained(output_dir)

# ذخیره توکنایزر
tokenizer.save_pretrained(output_dir)

print(f"مدل و توکنایزر آموزش‌دیده با موفقیت در مسیر '{output_dir}' ذخیره شدند.") # تغییر به print


مدل و توکنایزر آموزش‌دیده با موفقیت در مسیر 'roberta_sentiment_model_trained' ذخیره شدند.
